In [1]:
import pandas as pd
import numpy as np
import requests
from yahooquery import Ticker  # Step 2 (Speed)
import yfinance as yf          # Step 3 (Reliability)
import io
import json
import os
import time


In [2]:
import os
import json

# ==========================================
# CONFIGURATION
# ==========================================
# 1. FOLDER SETUP (The Fix for GitHub Portability)
DATA_FOLDER = "YfinanceDataDump"  # Relative path (creates folder in project root)

# Create the folder if it doesn't exist
if not os.path.exists(DATA_FOLDER):
    try:
        os.makedirs(DATA_FOLDER)
        print(f"Created data folder: {DATA_FOLDER}")
    except Exception as e:
        print(f"Warning: Could not create folder '{DATA_FOLDER}'. Saving to current directory. Error: {e}")
        DATA_FOLDER = "."

# 2. FILE PATHS (Everything saves inside the folder now)
CACHE_FILE = os.path.join(DATA_FOLDER, "financial_cache.json")
FORTRESS_CSV = os.path.join(DATA_FOLDER, "fortress_stocks.csv")
STRONG_CSV = os.path.join(DATA_FOLDER, "strong_stocks.csv")
RISKY_CSV = os.path.join(DATA_FOLDER, "risky_stocks.csv")
ANALYST_CSV = os.path.join(DATA_FOLDER, "Analyst_Fortress_Picks.csv")
BUFFETT_CSV = os.path.join(DATA_FOLDER, "Buffett_Value_Picks.csv")

# 3. UNIVERSE FILTERS
MIN_PRICE = 2.00               
MIN_VOLUME = 100_000          # 100K shares/day       
MIN_CAP = 50_000_000        # $50M
MIN_CURRENT_RATIO = 1.2
MAX_PE_RATIO = 100.0         

# 4. SAFETY THRESHOLDS 
MIN_INTEREST_COVERAGE = 1.5
MIN_ROIC = 0.05              # 5%
FORTRESS_MARGIN_THRESHOLD = 0.05  # 5%

EXCLUDED_SECTORS = ['Financial Services', 'Real Estate']
CACHE_EXPIRY_DAYS = 30 

# ==========================================
# HELPER FUNCTIONS
# ==========================================
def load_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, 'r') as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_cache(cache_data):
    try:
        with open(CACHE_FILE, 'w') as f:
            json.dump(cache_data, f)
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")

def calculate_altman_z_yfinance(bs, fin, market_cap):
    """
    Calculates Z-Score using yfinance DataFrames.
    Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E
    """
    try:
        # Helper to safely get value from Series
        def get_val(df, keys):
            for k in keys:
                if k in df.index:
                    return df.loc[k].iloc[0]
            return 0

        # Map yfinance row names
        total_assets = get_val(bs, ['Total Assets'])
        total_liab = get_val(bs, ['Total Liabilities Net Minority Interest', 'Total Liabilities'])
        current_assets = get_val(bs, ['Current Assets', 'Total Current Assets'])
        current_liab = get_val(bs, ['Current Liabilities', 'Total Current Liabilities'])
        retained_earnings = get_val(bs, ['Retained Earnings'])
        
        ebit = get_val(fin, ['EBIT', 'Operating Income'])
        total_revenue = get_val(fin, ['Total Revenue'])
        
        if total_assets == 0 or total_liab == 0: return 0

        # A: Working Capital / Total Assets
        A = (current_assets - current_liab) / total_assets
        
        # B: Retained Earnings / Total Assets
        B = retained_earnings / total_assets
        
        # C: EBIT / Total Assets
        C = ebit / total_assets
        
        # D: Market Value of Equity / Total Liabilities
        D = market_cap / total_liab
        
        # E: Sales / Total Assets
        E = total_revenue / total_assets
        
        return (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
    except Exception as e:
        return 0

In [3]:
# ==========================================
# STEP 1: FETCH CANADIAN UNIVERSE (ROBUST)
# ==========================================
def get_combined_universe():
    print("--- STEP 1: Fetching Canadian Universe (TSX & TSX-V) ---")
    tickers = []
    
    # --- METHOD 1: TMX OFFICIAL MOC LIST ---
    url_tmx = "https://www.tsx.com/files/trading/moc-eligible-stocks.txt"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        print("   -> Attempting to fetch official TMX list...")
        response = requests.get(url_tmx, headers=headers, timeout=10)
        response.raise_for_status()
        
        lines = response.content.decode('utf-8').split('\n')
        
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 3: continue
            
            # PARSING LOGIC: Detect if Exchange is at the Start or End
            # Format A: "TSX    RY    ROYAL BANK..."
            # Format B: "RY     ROYAL BANK...     TSX"
            
            symbol = None
            exchange = None
            
            if parts[0] in ['TSX', 'TSXV']:
                exchange = parts[0]
                symbol = parts[1]
            elif parts[-1] in ['TSX', 'TSXV']:
                exchange = parts[-1]
                symbol = parts[0]
            
            if symbol and exchange:
                # Clean Symbol (Yahoo uses hyphens, TMX uses dots)
                clean_symbol = symbol.replace('.', '-')
                
                if exchange == "TSX":
                    tickers.append(f"{clean_symbol}.TO")
                elif exchange == "TSXV":
                    tickers.append(f"{clean_symbol}.V")
                    
        tickers = list(set(tickers))
        print(f"   -> Success: Found {len(tickers)} stocks via TMX.")
        
    except Exception as e:
        print(f"   -> TMX Fetch Failed ({e}). Trying Backup...")

    # --- METHOD 2: WIKIPEDIA S&P/TSX COMPOSITE (If Method 1 returns 0 or fails) ---
    if len(tickers) == 0:
        try:
            print("   -> Attempting to scrape S&P/TSX Composite from Wikipedia...")
            url_wiki = "https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index"
            dfs = pd.read_html(url_wiki)
            
            # The constituent table is usually the first or second table
            for df in dfs:
                if 'Symbol' in df.columns:
                    # Wikipedia symbols often look like "RY" or "RY.TO"
                    wiki_tickers = df['Symbol'].astype(str).tolist()
                    for t in wiki_tickers:
                        t = t.replace('.', '-') # Fix BAM.A to BAM-A
                        if not t.endswith('.TO'):
                            t = t + ".TO"
                        tickers.append(t)
                    break
            
            tickers = list(set(tickers))
            print(f"   -> Success: Found {len(tickers)} stocks via Wikipedia.")
            
        except Exception as e:
            print(f"   -> Wikipedia Scraping Failed ({e}).")

    # --- METHOD 3: EMERGENCY FALLBACK LIST ---
    if len(tickers) == 0:
        print("   -> All web fetches failed. Using Emergency Hardcoded List.")
        tickers = [
            'SHOP.TO', 'RY.TO', 'TD.TO', 'CNR.TO', 'CP.TO', 'CSU.TO', 'ATD.TO', 'DOL.TO',
            'BMO.TO', 'BNS.TO', 'TRP.TO', 'ENB.TO', 'CNQ.TO', 'BCE.TO', 'CM.TO', 'MFC.TO',
            'QSR.TO', 'GIB-A.TO', 'SU.TO', 'WCN.TO', 'TECK-B.TO', 'T.TO', 'POW.TO', 'CVE.TO',
            'NA.TO', 'FTS.TO', 'EMA.TO', 'AEM.TO', 'WPM.TO', 'MRU.TO', 'OTEX.TO', 'SAP.TO',
            'L.TO', 'WN.TO', 'RCI-B.TO', 'CTC-A.TO', 'MG.TO', 'FM.TO', 'K.TO', 'CAE.TO',
            'TIH.TO', 'GIL.TO', 'DOO.TO', 'STN.TO', 'EFN.TO', 'KEY.TO', 'PPL.TO', 'IMO.TO'
        ]
        print(f"   -> Loaded {len(tickers)} major stocks.")

    return tickers

In [4]:
# ==========================================
# STEP 2: LIGHTWEIGHT SIEVE (YahooQuery)
# ==========================================
def get_initial_survivors(tickers):
    print(f"\n--- STEP 2: Running 'Lightweight' Filter on {len(tickers)} stocks ---")
    chunk_size = 500 
    survivors = []
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for i, chunk in enumerate(chunks):
        if i % 2 == 0: print(f" -> Processing Batch {i+1}/{len(chunks)}...")
        try:
            yq = Ticker(chunk, asynchronous=True)
            df_modules = yq.get_modules('summaryProfile summaryDetail financialData price defaultKeyStatistics')
            
            for symbol, data in df_modules.items():
                if isinstance(data, str): continue 
                try:
                    price = data.get('price', {}).get('regularMarketPrice', 0)
                    if price is None: price = 0
                    
                    vol = data.get('summaryDetail', {}).get('averageVolume', 0)
                    if vol is None or vol == 0:
                         vol = data.get('price', {}).get('averageDailyVolume10Day', 0)
                    
                    cap = data.get('price', {}).get('marketCap', 0)
                    if cap is None: cap = 0
                    
                    sector = data.get('summaryProfile', {}).get('sector', 'Unknown')
                    fin_data = data.get('financialData', {})
                    curr_ratio = fin_data.get('currentRatio', 0)
                    op_margins = fin_data.get('operatingMargins', 0)
                    if curr_ratio is None: curr_ratio = 0
                    if op_margins is None: op_margins = 0

                    # --- P/E RATIO CHECK ---
                    pe = data.get('summaryDetail', {}).get('trailingPE')
                    if pe is not None and pe > MAX_PE_RATIO: continue

                    # FILTERS
                    if price < MIN_PRICE: continue
                    if cap < MIN_CAP: continue
                    if vol < MIN_VOLUME: continue 
                    if any(x in sector for x in EXCLUDED_SECTORS): continue
                    if curr_ratio < MIN_CURRENT_RATIO: continue
                    if op_margins <= 0: continue 
                    
                    survivors.append({
                        'Ticker': symbol,
                        'Sector': sector,
                        'Price': price,
                        'Op Margin %': round(op_margins * 100, 2),
                        'P/E': round(pe, 2) if pe else 0,
                        'Curr Ratio': curr_ratio,
                        'Mkt Cap (B)': round(cap / 1_000_000_000, 2)
                    })
                except: continue
        except: continue
    return pd.DataFrame(survivors)

In [5]:
# ==========================================
# STEP 3: DEEP DIVE (yfinance + Cache)
# ==========================================
def get_advanced_metrics(survivor_df):
    tickers = survivor_df['Ticker'].tolist()
    print(f"\n--- STEP 3: Fetching Deep Financials for {len(tickers)} Survivors ---")
    
    cache = load_cache()
    current_time = time.time()
    expiry_seconds = CACHE_EXPIRY_DAYS * 86400
    
    final_data = []
    
    for i, ticker in enumerate(tickers):
        if i % 20 == 0: print(f" -> Analyzing {i+1}/{len(tickers)}: {ticker}...")
        
        # Helper: Logic to assign Tier based on Average Margin & Safety
        def determine_tier_history(metrics, is_fortress_margin, is_pos_margin):
            # 1. Safety Checks (Must pass these regardless of margins)
            if metrics['int_cov'] < MIN_INTEREST_COVERAGE: return "Risky"
            if metrics['roic'] < MIN_ROIC: return "Risky"
            
            # 2. Historical Margin Checks (Using the 4-Year Average)
            if is_fortress_margin: 
                return "Fortress"  # Avg Margin > 5%
            elif is_pos_margin:
                return "Strong"    # Avg Margin > 0%
            
            return "Risky"         # Avg Margin was negative

        # 1. CHECK CACHE
        cached_data = cache.get(ticker)
        # Note: We skip cache logic here to force a refresh on the first run with new logic
        # If you want to use cache, ensure 'avg_margin' logic is handled or clear old cache
        if cached_data and (current_time - cached_data['timestamp'] < expiry_seconds):
             # For now, we allow cache if it has valid data, but re-calculating margins 
             # usually requires the full dataframe. 
             # To force the new Average Margin check, we often proceed to fetch new data.
             pass 

        # 2. FETCH NEW DATA
        try:
            stock = yf.Ticker(ticker)
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty:
                cache[ticker] = {'timestamp': current_time, 'z_score': 0, 'roic': -999, 'int_cov': -999}
                continue
            
            # --- A. NEW LOGIC: 4-Year Average Margin Check ---
            try:
                # Get Operating Income (try 'Operating Income' first, then 'EBIT')
                if 'Operating Income' in fin.index:
                    op_income_history = fin.loc['Operating Income']
                elif 'EBIT' in fin.index:
                    op_income_history = fin.loc['EBIT']
                else:
                    op_income_history = pd.Series([0]) 

                # Get Revenue
                revenue_history = fin.loc['Total Revenue']
                
                # Calculate Margins for every available year
                # This automatically handles 1, 2, 3, or 4 years of data
                yearly_margins = (op_income_history / revenue_history).dropna()
                
                if len(yearly_margins) > 0:
                    avg_margin = yearly_margins.mean()
                    
                    # The Uniform Rule: Is the AVERAGE above the threshold?
                    is_fortress_margin = avg_margin > FORTRESS_MARGIN_THRESHOLD
                    is_positive_margin = avg_margin > 0
                else:
                    is_fortress_margin = False
                    is_positive_margin = False

            except Exception as e:
                # Fail safe
                is_fortress_margin = False
                is_positive_margin = False
            # ---------------------------------------------------

            # --- B. Standard Calculations (Safety Checks) ---
            def get_item(df, keys):
                for k in keys:
                    if k in df.index: return df.loc[k].iloc[0]
                return 0

            ebit = get_item(fin, ['EBIT', 'Operating Income', 'Pretax Income'])
            int_exp = get_item(fin, ['Interest Expense', 'Interest Expense Non Operating'])
            total_assets = get_item(bs, ['Total Assets'])
            curr_liab = get_item(bs, ['Current Liabilities', 'Total Current Liabilities'])
            
            # Interest Coverage
            int_exp = abs(int_exp)
            if int_exp == 0: int_cov = 100
            else: int_cov = ebit / int_exp
            
            # ROIC
            invested_cap = total_assets - curr_liab
            if invested_cap <= 0: roic = 0
            else: roic = ebit / invested_cap
            
            # Z-Score
            base_row = survivor_df[survivor_df['Ticker'] == ticker].iloc[0]
            mkt_cap_raw = base_row['Mkt Cap (B)'] * 1_000_000_000
            z = calculate_altman_z_yfinance(bs, fin, mkt_cap_raw)
            
            # Cache the metrics
            metrics = {
                'timestamp': current_time,
                'z_score': round(z, 2),
                'roic': roic,
                'int_cov': round(int_cov, 2)
            }
            cache[ticker] = metrics
            
            # --- C. Determine Final Tier ---
            tier = determine_tier_history(metrics, is_fortress_margin, is_positive_margin)

            final_data.append({
                'Ticker': ticker,
                'Tier': tier,
                'Price': base_row['Price'],
                'P/E': base_row['P/E'],
                'Sector': base_row['Sector'],
                'Z-Score': round(z, 2),
                'ROIC %': round(roic * 100, 2),
                'Op Margin %': base_row['Op Margin %'], # We still show the current TTM margin for reference
                'Avg Margin (4Y)': round(avg_margin * 100, 2) if 'avg_margin' in locals() else 0, # NEW COLUMN
                'Curr Ratio': base_row['Curr Ratio'],
                'Int Cov': round(int_cov, 2),
                'Mkt Cap (B)': base_row['Mkt Cap (B)']
            })
            
        except Exception as e:
            continue

    save_cache(cache)
    return pd.DataFrame(final_data)

In [6]:
# ==========================================
# MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    tickers = get_combined_universe()
    
    if len(tickers) > 0:
        survivors_df = get_initial_survivors(tickers)
        
        if not survivors_df.empty:
            print(f"\n✅ Step 2 Complete. {len(survivors_df)} stocks passed basic filters.")
            
            final_results = get_advanced_metrics(survivors_df)
            
            if not final_results.empty:
                final_results = final_results.sort_values(by=['Tier', 'Z-Score'], ascending=[True, False])
                
                # 1. Standard Split
                fortress_df = final_results[final_results['Tier'] == 'Fortress'].copy()
                strong_df = final_results[final_results['Tier'] == 'Strong'].copy()
                risky_df = final_results[final_results['Tier'] == 'Risky'].copy()
                
                # 3. Save Files (Updated to use Relative Paths from Cell 2)
                try:
                    fortress_df.to_csv(FORTRESS_CSV, index=False)
                    strong_df.to_csv(STRONG_CSV, index=False)
                    risky_df.to_csv(RISKY_CSV, index=False)
                    
                    print("\n" + "="*60)
                    print("RESULTS GENERATED")
                    print("="*60)
                    print(f"1. FORTRESS ({len(fortress_df)}): Saved to '{FORTRESS_CSV}'")
                    print(f"2. STRONG   ({len(strong_df)}): Saved to '{STRONG_CSV}'")
                    print(f"3. RISKY    ({len(risky_df)}): Saved to '{RISKY_CSV}'")
                except Exception as e:
                    print(f"\n⚠️ Error Saving Files: {e}")
                    print("Check if the file is open in Excel or if the folder exists.")
                
                pd.set_option('display.max_rows', 500)
                pd.set_option('display.max_columns', 20)
                pd.set_option('display.width', 1000)
                
                print("\n--- FORTRESS PREVIEW ---")
                print(fortress_df.head(15))
            else:
                print("No stocks passed the deep financial analysis.")
        else:
            print("No stocks passed the initial lightweight filter.")
    else:
        print("Could not fetch ticker universe.")

--- STEP 1: Fetching Canadian Universe (TSX & TSX-V) ---
   -> Attempting to fetch official TMX list...
   -> Success: Found 1088 stocks via TMX.

--- STEP 2: Running 'Lightweight' Filter on 1088 stocks ---
 -> Processing Batch 1/3...
 -> Processing Batch 3/3...

✅ Step 2 Complete. 111 stocks passed basic filters.

--- STEP 3: Fetching Deep Financials for 111 Survivors ---
 -> Analyzing 1/111: ATZ.TO...
 -> Analyzing 21/111: MRU.TO...
 -> Analyzing 41/111: CU.TO...
 -> Analyzing 61/111: TCL-A.TO...
 -> Analyzing 81/111: IMG.TO...
 -> Analyzing 101/111: LUN.TO...

RESULTS GENERATED
1. FORTRESS (74): Saved to 'YfinanceDataDump\fortress_stocks.csv'
2. STRONG   (5): Saved to 'YfinanceDataDump\strong_stocks.csv'
3. RISKY    (30): Saved to 'YfinanceDataDump\risky_stocks.csv'

--- FORTRESS PREVIEW ---
      Ticker      Tier   Price    P/E             Sector  Z-Score  ROIC %  Op Margin %  Avg Margin (4Y)  Curr Ratio  Int Cov  Mkt Cap (B)
103   WPM.TO  Fortress  161.36  53.61    Basic Materials

In [7]:
# 1. Define the function (if you haven't already in a previous cell)
def get_analyst_fortress_from_var(df_input):
    working_df = df_input.copy()
    tickers = working_df['Ticker'].tolist()
    
    print(f"\n--- STEP 4: Fetching Analyst Ratings for {len(tickers)} Stocks (From Memory) ---")
    print("    (Fetching serially to avoid throttling...)")
    
    analyst_data = []
    
    for i, ticker in enumerate(tickers):
        if i % 10 == 0: print(f" -> Analyst Scan {i+1}/{len(tickers)}: {ticker}...")
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            rec_mean = info.get('recommendationMean')
            target_price = info.get('targetMeanPrice')
            current_price = info.get('currentPrice')
            
            # Filter: Must be better than 2.0 (Lower is better)
            if rec_mean is None or rec_mean > 2.0: continue
            
            upside = 0
            if target_price and current_price:
                upside = round(((target_price - current_price) / current_price) * 100, 2)
            
            # Merge with existing data
            base_row = working_df[working_df['Ticker'] == ticker].iloc[0].to_dict()
            base_row['Analyst_Rating'] = rec_mean
            base_row['Target_Price'] = target_price
            base_row['Upside_%'] = upside
            
            analyst_data.append(base_row)
            time.sleep(0.2) # Polite delay
            
        except Exception:
            continue
            
    return pd.DataFrame(analyst_data)

# ==========================================
# 2. EXECUTE IT (Run this part!)
# ==========================================

# Check if fortress_df exists from the previous step
if 'fortress_df' in locals() and not fortress_df.empty:
    
    # Run the function
    Analyst_Fortress_DF = get_analyst_fortress_from_var(fortress_df)
    
    if not Analyst_Fortress_DF.empty:
        # Sort by best Analyst Rating (Lower is better) or Upside
        Analyst_Fortress_DF = Analyst_Fortress_DF.sort_values(by='Upside_%', ascending=False)
        
        # Display Results
        print("\n✅ Analyst Scan Complete!")
        print(f"Found {len(Analyst_Fortress_DF)} stocks with Buy Ratings (Score < 2.0)")
        
        # Save to CSV
        Analyst_Fortress_DF.to_csv(ANALYST_CSV, index=False)
        print("Saved to 'Analyst_Fortress_Picks.csv'")
        
        # Show top picks
        cols = ['Ticker', 'Price', 'Analyst_Rating', 'Upside_%', 'Target_Price', 'Tier']
        print(Analyst_Fortress_DF[cols].head(20))
    else:
        print("No stocks passed the Analyst filter.")
else:
    print("❌ 'fortress_df' not found or empty. Please run the Main Filter (Step 1-3) first.")


--- STEP 4: Fetching Analyst Ratings for 74 Stocks (From Memory) ---
    (Fetching serially to avoid throttling...)
 -> Analyst Scan 1/74: WPM.TO...
 -> Analyst Scan 11/74: DNG.TO...
 -> Analyst Scan 21/74: K.TO...
 -> Analyst Scan 31/74: RUS.TO...
 -> Analyst Scan 41/74: IMO.TO...
 -> Analyst Scan 51/74: CMG.TO...
 -> Analyst Scan 61/74: WN.TO...
 -> Analyst Scan 71/74: CTC-A.TO...

✅ Analyst Scan Complete!
Found 30 stocks with Buy Ratings (Score < 2.0)
Saved to 'Analyst_Fortress_Picks.csv'
     Ticker   Price  Analyst_Rating  Upside_%  Target_Price      Tier
17   VNP.TO   17.72         1.50000     43.23     25.381166  Fortress
21   CVE.TO   23.22         1.52941     27.81     29.676470  Fortress
3    TXG.TO   65.54         1.50000     26.58     82.958330  Fortress
24   PBH.TO  101.71         1.72727     25.04    127.181820  Fortress
20   PHX.TO    7.50         1.66667     22.22      9.166670  Fortress
6    AGI.TO   53.00         1.38462     21.25     64.265110  Fortress
23   DOO.TO 

In [8]:
import pandas as pd
from yahooquery import Ticker

# ==========================================
# BUFFETT "BELOW NAV" SCAN
# ==========================================
def get_buffett_value_picks(df_input):
    print(f"\n--- STEP 5: Warren Buffett 'Below NAV' Scan ---")
    print(f"    Scanning {len(df_input)} candidates for Deep Value...")
    print("    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%")

    tickers = df_input['Ticker'].tolist()
    buffett_candidates = []

    # Use YahooQuery for speed (Key Stats are summary data, no throttling risk here)
    chunk_size = 250
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            # We need defaultKeyStatistics (P/B) and financialData (ROE, Debt)
            data = yq.get_modules("defaultKeyStatistics financialData")

            for symbol in chunk:
                if isinstance(data, dict) and symbol in data:
                    try:
                        stats = data[symbol].get('defaultKeyStatistics', {})
                        fin = data[symbol].get('financialData', {})

                        # 1. Price to Book < 1.0 (The Core Rule)
                        pb = stats.get('priceToBook')
                        # Skip if None, > 1.0, or negative (insolvent)
                        if pb is None or pb >= 1.0 or pb <= 0: continue

                        # 2. Positive ROE (No Zombies)
                        roe = fin.get('returnOnEquity', 0)
                        if roe is None or roe <= 0: continue

                        # 3. Reasonable Debt (Safety)
                        # Buffett hates high leverage on weak companies
                        de = fin.get('debtToEquity', 0)
                        if de is None or de > 100: continue 

                        # Get base data from input_df
                        base_row = df_input[df_input['Ticker'] == symbol].iloc[0].to_dict()

                        # Add new Value Metrics
                        base_row['P/B Ratio'] = round(pb, 2)
                        base_row['ROE %'] = round(roe * 100, 2)
                        base_row['Debt/Eq %'] = round(de, 2)

                        buffett_candidates.append(base_row)

                    except: continue
        except: continue

    return pd.DataFrame(buffett_candidates)

# ==========================================
# EXECUTION BLOCK
# ==========================================
# Ensure we have the 'final_results' from the Main Filter
if 'final_results' in locals() and not final_results.empty:
    
    Buffett_Value_DF = get_buffett_value_picks(final_results)
    
    if not Buffett_Value_DF.empty:
        # Sort by P/B Ratio (Cheapest first)
        Buffett_Value_DF = Buffett_Value_DF.sort_values(by='P/B Ratio', ascending=True)
        
        # Save results
        Buffett_Value_DF.to_csv(BUFFETT_CSV, index=False)
        
        print("\n" + "="*60)
        print("BUFFETT SCAN COMPLETE")
        print("="*60)
        print(f"Found {len(Buffett_Value_DF)} Deep Value Stocks (Trading < Book Value)")
        print("Saved to: 'Buffett_Value_Picks.csv'")
        
        # Display
        pd.set_option('display.max_rows', 500)
        cols = ['Ticker', 'Price', 'P/B Ratio', 'ROE %', 'Debt/Eq %', 'Sector', 'Tier']
        print("\n--- DEEP VALUE PICKS ---")
        print(Buffett_Value_DF[cols].head(20))
        
    else:
        print("\n❌ No stocks passed the Buffett Value filter (All stocks are trading > Book Value).")
else:
    print("❌ 'final_results' variable not found. Please run the Main Filter first.")


--- STEP 5: Warren Buffett 'Below NAV' Scan ---
    Scanning 109 candidates for Deep Value...
    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%

❌ No stocks passed the Buffett Value filter (All stocks are trading > Book Value).


In [9]:
import pandas as pd
from yahooquery import Ticker
from IPython.display import display, Markdown

# ==========================================
# INSIDER FILTER FUNCTION
# ==========================================
def filter_for_insider_buying(tickers):
    print(f"🕵️ Scanning {len(tickers)} stocks for Insider Buying...")
    insider_picks = []
    
    # Chunk to prevent timeouts
    chunk_size = 20
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            df_insiders = yq.insider_transactions
            
            if isinstance(df_insiders, dict) or not hasattr(df_insiders, 'reset_index'): 
                continue
            
            df_insiders = df_insiders.reset_index()

            for symbol in chunk:
                if symbol not in df_insiders['symbol'].values:
                    continue
                
                stock_tx = df_insiders[df_insiders['symbol'] == symbol].copy()
                
                # Filter for Purchases
                buys = stock_tx[stock_tx['transactionText'].astype(str).str.contains("Purchase", case=False, na=False)]
                sells = stock_tx[stock_tx['transactionText'].astype(str).str.contains("Sale", case=False, na=False)]
                
                # Logic: Net Positive Buying
                buy_vol = buys['shares'].sum() if not buys.empty else 0
                sell_vol = sells['shares'].sum() if not sells.empty else 0
                
                if buy_vol > sell_vol:
                    insider_picks.append({
                        'Ticker': symbol,
                        'Insider_Buys_Count': len(buys),
                        'Net_Shares_Bought': buy_vol - sell_vol
                    })
                        
        except Exception:
            continue

    return pd.DataFrame(insider_picks)

# ==========================================
# 2. CREATE 'Fortress_insiders' DATAFRAME
# ==========================================

# Use fortress_df if it exists, otherwise use the top 20 backup list
if 'fortress_df' in locals() and not fortress_df.empty:
    target_tickers = fortress_df['Ticker'].tolist()
else:
    # Backup list from your logs so this runs even if you restarted the kernel
    target_tickers = [
        'PET.TO', 'MFI.TO', 'TXG.TO', 'SAP.TO', 'PAAS.TO', 'NEO.TO', 'WPM.TO', 
        'FNV.TO', 'LUG.TO', 'DPM.TO', 'ASM.TO', 'PNG.V', 'DSG.TO', 'KNT.TO', 
        'GGD.TO', 'GRGD.TO', 'WDO.TO', 'OGC.TO', 'DNG.TO', 'CLS.TO'
    ]

# Run the filter and assign to the variable you requested
Fortress_insiders = filter_for_insider_buying(target_tickers)

# Display so Data Wrangler picks it up
print(f"✅ Created 'Fortress_insiders' with {len(Fortress_insiders)} rows.")
display(Fortress_insiders)

🕵️ Scanning 74 stocks for Insider Buying...
✅ Created 'Fortress_insiders' with 0 rows.


""


In [10]:
import pandas as pd
from yahooquery import Ticker

# ==========================================
# Michael EV/EBITDA SCAN
# ==========================================
def get_buffett_value_picks(df_input):
    print(f"\n--- STEP 5: Michael EV/EBITDA Scan ---")
    print(f"    Scanning {len(df_input)} candidates for Deep Value...")
    print("    Criteria: EV/EBITDA < 10 | ROE > 0% (Profitable) | Debt/Eq < 100%")

    tickers = df_input['Ticker'].tolist()
    buffett_candidates = []

    # Use YahooQuery for speed (Key Stats are summary data, no throttling risk here)
    chunk_size = 250
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            # We need defaultKeyStatistics (P/B) and financialData (ROE, Debt)
            data = yq.get_modules("defaultKeyStatistics financialData")

            for symbol in chunk:
                if isinstance(data, dict) and symbol in data:
                    try:
                        stats = data[symbol].get('defaultKeyStatistics', {})
                        fin = data[symbol].get('financialData', {})

                        # 1. Price to Book < 1.0 (The Core Rule)
                        pb = stats.get('priceToBook')
                        # Skip if None, > 1.0, or negative (insolvent)
                        if pb is None or pb >= 1.0 or pb <= 0: continue

                        # 2. Positive ROE (No Zombies)
                        roe = fin.get('returnOnEquity', 0)
                        if roe is None or roe <= 0: continue

                        # 3. Reasonable Debt (Safety)
                        # Buffett hates high leverage on weak companies
                        de = fin.get('debtToEquity', 0)
                        if de is None or de > 100: continue 

                        # Get base data from input_df
                        base_row = df_input[df_input['Ticker'] == symbol].iloc[0].to_dict()

                        # Add new Value Metrics
                        base_row['P/B Ratio'] = round(pb, 2)
                        base_row['ROE %'] = round(roe * 100, 2)
                        base_row['Debt/Eq %'] = round(de, 2)

                        buffett_candidates.append(base_row)

                    except: continue
        except: continue

    return pd.DataFrame(buffett_candidates)

# ==========================================
# EXECUTION BLOCK
# ==========================================
# Ensure we have the 'final_results' from the Main Filter
if 'final_results' in locals() and not final_results.empty:
    
    Buffett_Value_DF = get_buffett_value_picks(final_results)
    
    if not Buffett_Value_DF.empty:
        # Sort by P/B Ratio (Cheapest first)
        Buffett_Value_DF = Buffett_Value_DF.sort_values(by='P/B Ratio', ascending=True)
        
        # Save results
        Buffett_Value_DF.to_csv(BUFFETT_CSV, index=False)
        
        print("\n" + "="*60)
        print("BUFFETT SCAN COMPLETE")
        print("="*60)
        print(f"Found {len(Buffett_Value_DF)} Deep Value Stocks (Trading < Book Value)")
        print("Saved to: 'Buffett_Value_Picks.csv'")
        
        # Display
        pd.set_option('display.max_rows', 500)
        cols = ['Ticker', 'Price', 'P/B Ratio', 'ROE %', 'Debt/Eq %', 'Sector', 'Tier']
        print("\n--- DEEP VALUE PICKS ---")
        print(Buffett_Value_DF[cols].head(20))
        
    else:
        print("\n❌ No stocks passed the Buffett Value filter (All stocks are trading > Book Value).")
else:
    print("❌ 'final_results' variable not found. Please run the Main Filter first.")


--- STEP 5: Michael EV/EBITDA Scan ---
    Scanning 109 candidates for Deep Value...
    Criteria: EV/EBITDA < 10 | ROE > 0% (Profitable) | Debt/Eq < 100%

❌ No stocks passed the Buffett Value filter (All stocks are trading > Book Value).


In [11]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================



import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND','ARCC','BANC','ONB','TCL-A.TO','ADMA','MIR','APG','SEI','FLEX','DD','MATR.TO'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 12 stocks ---
1. Fetching Analyst Ratings from Finviz...
   Skipping Finviz for TCL-A.TO: HTTP error for URL https://finviz.com/quote.ashx?t=TCL-A.TO: 404 Client Error: Not Found for url: https://finviz.com/quote.ashx?t=TCL-A.TO
   Skipping Finviz for MATR.TO: HTTP error for URL https://finviz.com/quote.ashx?t=MATR.TO: 404 Client Error: Not Found for url: https://finviz.com/quote.ashx?t=MATR.TO
2. Fetching Price & Volatility from yfinance...


C:\Users\James\AppData\Local\Temp\ipykernel_27032\579518594.py:46: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)



--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
6,GRND,13.54,0.97,17.62,-46.12,1.40,21.75,0.83,3.18
7,MATR.TO,7.98,0.63,10.64,-38.57,NaN,NaN,0.71,3.07
0,ADMA,18.24,-0.65,17.91,-28.94,1.00,30.00,0.76,3.26
8,MIR,23.42,-1.18,19.97,-22.65,1.12,30.62,0.71,3.49
10,SEI,45.97,-0.28,32.67,-19.41,1.17,65.45,0.69,5.90
5,FLEX,60.42,-2.03,48.48,-16.34,1.50,76.00,0.31,2.91
11,TCL-A.TO,22.72,0.09,18.98,-11.42,NaN,NaN,0.63,1.85
2,ARCC,20.23,-0.30,20.48,-9.64,1.27,22.64,0.89,1.38
9,ONB,22.31,-1.33,21.41,-6.56,1.85,25.92,0.76,2.14
1,APG,38.26,-1.54,31.52,-5.72,1.45,43.40,1.01,1.93


In [12]:
import requests
import pandas as pd
import google as genai
import enum
from typing_extensions import TypedDict
import json
import plotly.express as px
import sys
#!"{sys.executable}" -m pip install google.genai
#!"{sys.executable}" -m pip install plotly.express

In [13]:
tickers_gemini = ['MATR.TO'] 

In [14]:
import os
from google import genai
from google.genai import types
from IPython.display import display, Markdown

# ==========================================
# SECURE CONFIGURATION
# ==========================================

# 1. Define the path to your key file
# If the file is in the same folder as this notebook, just use the filename.
KEY_FILE_PATH = "C:\\Users\\James\\OneDrive - McMaster University\\Gemini API Key\\gemini_key.txt"

def load_api_key(filepath):
    """
    Reads the API key from a local file to avoid hardcoding it.
    """
    try:
        with open(filepath, "r") as f:
            # .strip() removes any accidental newlines or spaces
            return f.read().strip()
    except FileNotFoundError:
        print(f"❌ Error: Could not find the file '{filepath}'")
        print("Please create a text file with your API key in it.")
        return None
    except Exception as e:
        print(f"❌ Error reading key file: {e}")
        return None

# 2. Load the key and set the environment variable
api_key = load_api_key(KEY_FILE_PATH)

if api_key:
    os.environ["GEMINI_API_KEY"] = api_key
    print("✅ API Key loaded securely.")
else:
    print("⚠️ CRITICAL: API Key not loaded. The script will fail.")

# ==========================================
# SENTIMENT ANALYSIS FUNCTION
# ==========================================
def analyze_sentiment_gemini_3(tickers_gemini, company_name=None):
    """
    Uses Gemini 3 Flash (Preview) with 'High' Thinking Level and Google Search 
    using the NEW google-genai SDK.
    """
    if not os.environ.get("GEMINI_API_KEY"):
        print("❌ Stop: No API Key found.")
        return

    print(f"\n🧠 Gemini 3 is thinking (High Reasoning Mode)... analyzing ${tickers_gemini}...")

    # Initialize Client
    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            include_thoughts=False, 
            thinking_level="HIGH"
        ),
        tools=[types.Tool(
            google_search=types.GoogleSearch() 
        )],
        response_modalities=["TEXT"]
    )

    prompt = f"""
    You are a Senior Equity Research Analyst using the Gemini 3 Reasoning Engine. 
    Perform a deep "Market Sentiment Analysis" on {tickers_gemini} ({company_name if company_name else 'the company'}).
    
    Step 1: SEARCH. Use Google Search to find the latest (last 30 days) news, analyst notes, and SEC filings.
    Step 2: REASON. Analyze the search results to determine the true market psychology. Look for contradictions between price action and news.
    
    Investigate these 4 Pillars:
    1. **News Virality**: Are headlines fear-mongering or euphoric? (Look for scandals, lawsuits, or product breakthroughs).
    2. **Analyst Shifts**: Are price targets moving UP or DOWN in the last week?
    3. **Institutional Flows**: Any reports of hedge funds or insiders buying/selling?
    4. **The "Whisper" Number**: What are traders saying on forums vs. official guidance?

    **OUTPUT FORMAT:**
    Produce a professional Markdown report:
    
    ## 🧠 Gemini 3 Sentiment Report: {tickers_gemini}
    **Reasoning Depth:** High
    **Sentiment Score:** [1-10]
    **Verdict:** [Buy / Hold / Sell / Speculative]
    
    ### 1. The Bull Thesis (Why it goes up)
    * ...
    
    ### 2. The Bear Thesis (Why it goes down)
    * ...
    
    ### 3. Deep Dive Analysis
    * **News Analysis**: ...
    * **Smart Money**: ...
    * **Financial Statement Analysis**: (Historic performance over last 3 years + expected performance)
    
    ### 4. Conclusion
    [Summary of whether the current price is a trap or an opportunity]
    """

    try:
        response = client.models.generate_content(
            model='gemini-3-flash-preview', # Or 'gemini-3-flash-preview'
            contents=prompt,
            config=config
        )
        display(Markdown(response.text))
        
    except Exception as e:
        print(f"❌ Error: {e}")

# ==========================================
# EXECUTION
# ==========================================
# Only run this if the key loaded successfully
if os.environ.get("GEMINI_API_KEY"):
    analyze_sentiment_gemini_3(tickers_gemini, tickers_gemini)

✅ API Key loaded securely.

🧠 Gemini 3 is thinking (High Reasoning Mode)... analyzing $['MATR.TO']...


The following report is based on the **Gemini 3 Reasoning Engine** analysis of **Mattr Corp (MATR.TO)**—formerly Shawcor Ltd.—evaluated as of January 1, 2026.

---

## 🧠 Gemini 3 Sentiment Report: ['MATR.TO']
**Reasoning Depth:** High  
**Sentiment Score:** 3.8 / 10  
**Verdict:** Speculative / Contrarian Hold  

### 1. The Bull Thesis (Why it goes up)
*   **Infrastructure Pure-Play:** Having successfully divested its legacy pipe-coating business (finalized with the Thermotite sale in June 2025), Mattr is now a leaner, high-margin materials technology company focused on electrification and water management.
*   **Connection Technologies Growth:** The acquisition of AmerCable has doubled revenue in the Connection segment, positioning Mattr as a critical supplier for North American industrial electrification and mining.
*   **Institutional "Diamond Hands":** Top-tier value funds like **Turtle Creek Asset Management** and **EdgePoint** remain significant shareholders (combined ~25%+), signaling long-term conviction in the management's transformation strategy.
*   **Insider Buying:** Recent filings show meaningful open-market purchases by the Chairman and other directors in the $7.00–$9.00 range, suggesting a perceived floor by those closest to the operations.

### 2. The Bear Thesis (Why it goes down)
*   **Leverage Red Flag:** The Net Debt to Adjusted EBITDA ratio has spiked to **3.86x** following the AmerCable acquisition, well above management’s 2.0x target. In a high-interest-rate environment, this leverage is the primary driver of the current "Sell" sentiment.
*   **Guidance Trauma:** The stock suffered a **21.36% single-day crash** in November 2025 following a significant Q3 earnings miss ($0.05 EPS vs. $0.12 expected) and a cautious outlook for 2026.
*   **Cyclical Slowdown:** Demand for the "Composite Technologies" segment (Flexpipe) is softening as oilfield activity levels decline, neutralizing the growth gains seen in the cabling division.
*   **Capex Retrenchment:** Management has lowered its CAPEX guidance, suggesting a shift from "growth mode" to "survival/debt-repayment mode."

### 3. Deep Dive Analysis
#### **News Analysis**
The news cycle for MATR.TO is currently dominated by **"Post-Earnings Fatigue."** While headlines in early 2025 were euphoric regarding the AmerCable integration, the narrative has shifted to "Tariff Uncertainty" and "Canadian Industrial Slowdown." There is a visible contradiction: Revenue is surging (+39% YoY), yet the stock is hitting 52-week lows. This indicates the market no longer cares about top-line growth and is hyper-focused on **free cash flow** and **deleveraging.**

#### **Smart Money**
*   **Institutions:** Total institutional ownership remains high (approx. 45%), but there has been a 6.6% decline in total institutional positions in the last quarter of 2025.
*   **Whisper Number:** Retail traders on forums (CEO.ca/Stockhouse) are increasingly frustrated, calling the stock a "value trap" until the debt ratio drops below 3.0x. The "Whisper" for Q4 2025 is pessimistic, with traders expecting further margin compression.

#### **Financial Statement Analysis**
*   **Historic Performance (Last 3 Years):** Mattr underwent a total identity shift. It shed low-margin "Pipeline Performance" assets (Shawcor) to focus on Composites. This led to a temporary boost in the share price to $15+ in 2024.
*   **Current/Expected Performance:** The 2025 fiscal year has been defined by "Acquisition Indigestion." While gross margins remain healthy at ~25%, the interest expense on the US$280M AmerCable debt is eating the bottom line. Analysts have aggressively cut price targets from $15.00 down to **$7.00 - $9.00.**

### 4. Conclusion
The current price of **MATR.TO** is a **liquidity-driven trap** for short-term traders but a **high-risk opportunity** for deep-value investors. The 21% post-earnings plunge has baked in much of the "worst-case" debt scenario. However, until management provides a concrete timeline for reaching their 2.0x leverage target or announces a major asset sale to pay down the AmerCable loan, the stock is likely to trade sideways or slightly lower.

**Final Tactical Note:** Monitor the **NCIB (Buyback)** activity. If management pauses buybacks to prioritize debt repayment, it will be a short-term bearish signal for the price but a long-term bullish signal for the balance sheet's health.